In [40]:
import os, csv, json, re
import pandas as pd
import numpy as np

In [ ]:
df_final = pd.read_pickle("predicted_output.pkl")
df_final.columns

In [ ]:
df_final

In [43]:
# # run this cell to combine two halves into one again, no need to combine the input feature, only the output points
# df_final_top_half = pd.read_pickle("data_predicted_topHalf_time3600_perRun150.pkl")
# df_final_bottom_half = pd.read_pickle("data_predicted_bottomHalf_time3600_perRun150.pkl")

In [44]:
# This is a unit testing cell that just makes assertions, no outcome results from this 

# y_test = np.asarray(df_final.y_test.tolist())
# num_columns = 22  # Number of columns (pointType_<num>)
# 
# # Reshape y_test back to a 3D array
# arr_3d = y_test.reshape(y_test.shape[0], num_columns, 2)
# # Create a dictionary to hold column names and data
# 
# column_names = [f"pointType_{i}" for i in range(num_columns)]
# # print(column_names)
# data_dict = {col: arr_3d[:, i, :].tolist() for i, col in enumerate(column_names)}
# 
# # Create the DataFrame
# df_reversed = pd.DataFrame(data_dict)
# 
# #assert that the reversing is correct
# assert df_reversed.equals(df_final[[col for col in df_final.columns if 'pointType' in col]])
# 
# # assertion to check that the labels match original data accurately
# arr = df_final[[col for col in df_final.columns if 'pointType' in col]].values
# arr_np = np.asarray(arr.tolist())
# y_test = arr_np.reshape(arr_np.shape[0], arr_np.shape[1] * arr_np.shape[2])
# np.testing.assert_array_equal(y_test,np.asarray(df_final.y_test.tolist()))

In [45]:
# This is the cell that produce the predicted points

y_prediction = np.asarray(df_final.predictions.tolist())

#in case of halving use the following
# y_prediction = np.concatenate((np.asarray(df_final_top_half.predictions.tolist()),np.asarray(df_final_bottom_half.predictions.tolist())), axis=1)

num_columns = 22  # Number of columns (pointType_<num>)

# Reshape y_test back to a 3D array
arr_3d = y_prediction.reshape(y_prediction.shape[0], num_columns, 2)
# Create a dictionary to hold column names and data

column_names = [f"pointType_{i}" for i in range(num_columns)]
#in case of halving use the following
# column_names = ['pointType_21', 'pointType_0', 'pointType_1',
#        'pointType_2', 'pointType_3', 'pointType_4', 'pointType_5',
#        'pointType_6', 'pointType_7', 'pointType_8', 'pointType_9',
#        'pointType_10', 'pointType_11', 'pointType_12', 'pointType_13',
#        'pointType_14', 'pointType_15', 'pointType_16', 'pointType_17',
#        'pointType_18', 'pointType_19', 'pointType_20']
# print(column_names)
data_dict = {col: arr_3d[:, i, :].tolist() for i, col in enumerate(column_names)}

# Create the DataFrame
df_reversed = pd.DataFrame(data_dict)


In [ ]:
df_final[(df_final.participant_number == "73") & (df_final.sample_number == "1421")]

In [47]:
df_final = df_final[[col for col in df_final.columns if not 'pointType' in col]]
df_final = pd.concat([df_final, df_reversed], axis=1)

In [58]:
def save_sample(df, participant_number, left_or_right, sample_number):
    sampleDir = os.path.normpath("data2/" + participant_number + "/feet/" + left_or_right + "/samples/" + sample_number)
    
    # Create the directory if it doesn't exist
    # os.makedirs(sampleDir, exist_ok=True)
    print(participant_number, left_or_right, sample_number)
    # Convert DataFrame to a list of dictionaries (each dictionary represents a point)
    points_data = []
    for row in df.items():
        # print(row[0],row[1])
        points_data.append({"points": row[1], "pointType": int(re.findall(r'\d+', row[0])[0])})
        # print(points_data)
    # print(df)
    
    # print(points_data)
    # Write the list of dictionaries to the points.json file
    points_file = os.path.join(sampleDir, "predictedPoints.json")
    print(points_file)
    with open(points_file, 'w') as file:
        json.dump(points_data, file, indent=2)  # Indent for readability

In [ ]:
# Assuming you already have df_final containing your processed data
# You can replace this with your actual DataFrame
for _, row in df_final.iterrows():
    participant_number = row['participant_number']
    left_or_right = row['left_or_right']
    sample_number = row['sample_number']
    selected_columns = [col for col in df_final.columns if col.startswith("pointType_")]
    df_selected = row[selected_columns]

    save_sample(df_selected, participant_number, left_or_right, sample_number)
